In [1]:
import sys

In [2]:
sys.path.insert(0, '../..')

In [3]:
from sqlalchemy import func

In [4]:
import pandas as pd

In [5]:
from database import session_scope
from database.models.forwarding_events import ForwardingEvents
from database.models.open_channels import OpenChannels

2019-02-28 19:02:30 [debug    ] getting label                  label=LPU_PGDATABASE
2019-02-28 19:02:30 [debug    ] getting label                  label=LPU_PGUSER
2019-02-28 19:02:30 [debug    ] getting label                  label=LPU_PGPASSWORD
/Users/pierre/src/lightning-power-users-website/venv/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [6]:
pd.options.display.float_format = '{:,}'.format
pd.options.display. = '{:,}'.format


In [7]:
with session_scope() as session:
    query = (
        session
        .query(
                ForwardingEvents.chan_id_out, 
            
                OpenChannels.remote_pubkey,
               OpenChannels.local_balance, 
               OpenChannels.remote_balance, 
               func.count(ForwardingEvents.timestamp).label('count'), 
               func.sum(ForwardingEvents.amt_out).label('amt_out'), 
               func.sum(ForwardingEvents.fee).label('fee'))
        .group_by(OpenChannels, ForwardingEvents.chan_id_out)
        .order_by(func.sum(ForwardingEvents.amt_out).desc())
        .join(OpenChannels, ForwardingEvents.chan_id_out == OpenChannels.chan_id)
    )
    df = pd.read_sql(query.statement, session.bind)

In [9]:
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>chan_id_out</th>\n      <th>remote_pubkey</th>\n      <th>local_balance</th>\n      <th>remote_balance</th>\n      <th>count</th>\n      <th>amt_out</th>\n      <th>fee</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>619583598392180737</td>\n      <td>036a54f02d2186de192e4bcec3f7b47adb43b1fa965793...</td>\n      <td>10560679</td>\n      <td>6212515</td>\n      <td>79</td>\n      <td>35,307,053.0</td>\n      <td>107.0</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>618903000782012417</td>\n      <td>03864ef025fde8fb587d989186ce6a4a186895ee44a926...</td>\n      <td>349824</td>\n      <td>15646159</td>\n      <td>12</td>\n      <td>15,755,634.0</td>\n      <td>26.0</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>611476899153707008</td>\n      <td>03be0a28c386a4dbd43535e615e1b370995fa57fc8be42...</td>\n      <td>5865344</td>\n      <t